In [1]:
import pandas as pd

df = pd.read_csv('./df_50.csv')


In [2]:
import numpy as np
import time
import cv2
import urllib

imgGraySmall = []

for m in range(len(df['imgURL'])):
    res = urllib.request.urlopen(df['imgURL'][m])
    i = np.asarray(bytearray(res.read()), dtype="uint8")
    imgGray = cv2.imdecode(i, cv2.IMREAD_GRAYSCALE)
    imgResize = cv2.resize(imgGray, (100, 100), interpolation=cv2.INTER_AREA)
    imgGraySmall.insert(m,imgResize)
    time.sleep(1)

In [3]:
tfR = []
for n in range(len(imgGraySmall)): 
    imgGraySmall[n][ imgGraySmall[n] == np.argmax(np.bincount(imgGraySmall[n].flatten()))]= 255 #找出背景最多的數字，再指定成255
    img90 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_90_CLOCKWISE)
    img180 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_180)
    img270 = cv2.rotate(imgGraySmall[n], cv2.ROTATE_90_COUNTERCLOCKWISE)
    tfR.extend([imgGraySmall[n], img90, img180, img270])

tfF = []
for o in range(len(tfR)):
    imgUD = cv2.flip(tfR[o], 0)
    tfF.extend([tfR[o], imgUD])        

tfN = []
for p in range(len(tfF)):
    img_negative = 255-tfF[p]
    img_negative[ img_negative == np.argmax(np.bincount(img_negative.flatten()))]= 255
    tfN.extend([tfF[p], img_negative])

In [4]:
flatlist = []
for q in tfN:
    flatlist.append(q.flatten())

In [5]:
aaa = pd.DataFrame(flatlist).astype('uint8')

In [6]:
namelist = []
for r in df['productID']:
    for s in range(16):
        namelist.append(r)

In [7]:
aaa.index = namelist

In [12]:
aaa.to_csv('transformation.csv', encoding = 'utf8')